In [ ]:
#!/usr/bin/env python
# coding: utf-8

In [ ]:
import codecs
from datetime import datetime as dt
import json
import sys
import numpy as np
import os
import pandas as pd
import plotly
from plotly import subplots
import plotly.express as px
import plotly.tools as tls
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as offline
import re
import sys
if "ipy" in sys.argv[0]:
    offline.init_notebook_mode()
from cov19utils import create_basic_plot_figure, \
    show_and_clear, moving_average, \
    blank2zero, csv2array, \
    get_twitter, tweet_with_image, \
    get_gpr_predict, FONT_NAME, DT_OFFSET, \
    download_if_needed, json2nparr, code2int, age2int, \
    get_populations, get_os_idx_of_arr, dump_val_in_arr, \
    calc_last1w2w_dif, create_basic_scatter_figure, \
    show_and_save_plotly
import re
import requests
from sklearn.linear_model import LinearRegression

In [ ]:
u = "https://www.city.sapporo.jp/hokenjo/f1kansen/"
f = "2019n-covhassei.html"
download_if_needed(u, f)

In [ ]:
with codecs.open(f, encoding='UTF-8') as fin:
    for line in fin:
        if '更新日' in line:
            m = re.search(r"更新日：(\d+)年(\d+)月(\d+)日", line)
            ymd = m.groups()
            update = "{}/{}/{}".format(ymd[0], ymd[1], ymd[2])
            print(update)

In [ ]:
with codecs.open("sapporo.tmp", encoding='UTF-8') as fin:
    last_updated = fin.read().strip()
print(last_updated)

In [ ]:
if last_updated == update:
    print("maybe the same html, nothing to do.")
    if "ipy" in sys.argv[0]:
        pass#exit()
    else:
        sys.exit()

In [ ]:
df = pd.read_html(f)

In [ ]:
posit = float(df[3]['陽性率'][0].replace('％', ''))
tests = int(df[3]['新規検査人数合計'][0].replace('人', '').replace(',', ''))
cases = int(df[3]['本日の陽性者数'][0].replace('人', ''))
#print("p:{} t:{} c:{}".format(posit, tests, cases))

In [ ]:
d = df[4]['死亡（累計）'][0]
deaths = 0
m = re.search(r'\+(\d+)人', d)
if m:
    deaths = int(m.groups()[0])
print("deaths: {} (d)".format(deaths, d))

In [ ]:
t = df[4]['陽性者数（累計）'][2]
m = re.search(r'\+(\d+)人', t)
tracked = int(m.groups()[0])
#print("tracked: {}".format(tracked))
untracked = round(100.0 * (cases - tracked) / cases, 2)
#print("untracked: {}".format(untracked))

In [ ]:
print("pos:{}% test:{} case:{} death:{} track:{} untrack:{}%".format(
    posit, tests, cases, deaths, tracked, untracked))

In [ ]:
l = "{},{},{},{},{}\n".format(update, tests, cases, tracked, deaths)
print(l)
with open("csv/sapporo.csv", "a") as fout:
    fout.write(l)

In [ ]:
curr = pd.read_csv("csv/sapporo.csv", encoding='shift-jis', header=0)
n = len(curr)
print(n)

In [ ]:
prev = 0
with codecs.open("csv/sapporo.tmp", encoding="utf-8") as f:
    prev = f.read().strip()
prev = int(prev)
print(prev)

In [ ]:
if n == prev:
    print("maybe the same data, nothing to do.")
    if "ipy" in sys.argv[0]:
        pass#exit()
    else:
        sys.exit()

In [ ]:
curr['untrack'] = ((curr['postive'] - curr['trace']) / curr['postive']) * 100
curr['posrate'] = (curr['postive'] / curr['test']) * 100
curr['ma_posr'] = moving_average(curr['posrate'] )

In [ ]:
df = curr

In [ ]:
titles = ["検査数","陽性者","経路不明率[%]","陽性率[%]", "死者数", "陽性率移動平均[%]"]
cols = ['test', 'postive', 'untrack', 'posrate', 'death', 'ma_posr']
fig = subplots.make_subplots(shared_xaxes=True,
    rows=3, cols=2, subplot_titles=titles,
    horizontal_spacing=0.05, vertical_spacing=0.07)
i = 0
for c in cols:
    if c == 'death':
        trace = go.Bar(x=df['date'], y=df[c], name=titles[i])
        fig.add_trace(trace, 1 + int(i/2), 1 + (i % 2))
    else:
        trace = go.Scatter(x=df['date'], y=df[c], name=titles[i])
        fig.add_trace(trace, 1 + int(i/2), 1 + (i % 2))
        fig.update_yaxes(range=[0, (max(df[c])*1.1)], row=1 + int(i/2), col=1 + (i % 2))
    i += 1

today_str = dt.now().isoformat()[:16].replace('T', ' ')
title = '札幌感染状況 2021-04-25 ～ {}'.format(today_str)
fig.update_layout(
    template='plotly_dark',
    margin={"r":10,"t":80,"l":10,"b":100},
    height=700, showlegend=False, width=600,
    title=title)
show_and_save_plotly(fig, "sapporo.jpg", js=False, show=True, image=True, html=True)

In [ ]:
tw = get_twitter()
tweet_with_image(tw, "docs/images/sapporo.jpg", title)

In [ ]:
with open("csv/sapporo.tmp", "w") as f:
    f.write("{}".format(n))

In [ ]:
with open("sapporo.tmp", "w") as f:
    f.write("{}".format(update))

In [ ]:
print(title)